<img src="./images/Konane.JPG">

# CA 2 (Adversial search: Game)
In this project, we try to develop an intelligent agent for playing a board game. <br>
This game is deterministix and zero sum. So for solving this problem, we use minimax algorithm. <br>
Depth of state space is very large and because of that we can't run this algorithm till leaves of tree. We should do a depth-limited search and use evaluation function for last depth nodes. This function is some how utility function that tells how good a state is. This function make diffrences in this types of agents. <br>
For deeper searchs, we should prune some branches. For this purpose, we use alpha-beta pruning algorithm. <br>
We discuss about diffrent parts in below.

## Minimax player
We should define a new player that is our agent. It inherits from __Game__ and __Player__ class and extends their functionallities. This player class has five main funcitons:

### initialize(self, side, depth)
It's overriden method of player class. It sets main features of player instance. 
(__name__, 
__side__: Player's color,
__depth__: _Show maximum depth that minimax explores it._
)

In [1]:
def initialize(self, side, depth):
    self.name = "Minimax"
    self.side = side
    self.depth = depth

### getMove(self, board)
It's another overriden method of player class that returns player's next move. Game class deal with function to handle player movements. It takes current board configuration and returns player's next move. <br>
In this function we create level 1 nodes of minimax tree and behave like a maximizer node because we want to maximize our point. We discuss about minimax in details in next section. It do alpha-beta pruning too. <br> 
This function aggregate best possible move that get from belower nodes in minimax tree.

In [2]:
def getMove(self, board):
    best_move_score = float(-inf)
    initial_alpha = float(-inf)
    initial_beta = float(inf)
    moves = self.generateMoves(board, self.side)
    if not moves:
        return []
    for move in moves:
        next_board = self.nextBoard(board, self.side, move)
        next_board_score = self.minValue(next_board, initial_alpha, initial_beta, 1)
        if next_board_score > best_move_score:
            best_move = move
            best_move_score = next_board_score
        if best_move_score > initial_beta:
            return best_move_score
        initial_alpha = max(initial_alpha, best_move_score)
    return best_move

## Minimax algorithm backbones
### minValue(self, board, alpha, beta, depth)   &   maxValue(self, board, alpha, beta, depth)
This two functions implement minimax algorithm main functionallity. In a zero sum game, our opponent try to decrease our point and we try to increase our point. If we have tree that models our move and board states, we can iterate on it to find win scenario and select our moves. Each terminal state has a point that define the point that we get when we reach to this tate at the end of the game. Each terminal state tell it's point to it's parrent. Middle nodes are minimizer nodes or maximizer nodes. In one turn (level), we try to achieve best score and in next our opponent try to increase it's score and because game is zero sum, it decrease our score. So we are maximizer nodes and our opponent or children in minimax tree are minimizer nodes. In maximizer nodes, we calculate maximum values of our children. Minimizer nodes calculate minimum of it's children. <br>
In the first part of this function, we see depth limiting. We should make desicion in reasonable time. Because of resource limits, we should do depth limited search. <br>
In the last part we can see implemenation of alpha beta prunning. This algorithm prune non promising branches and block them to save time to explore deeper. <br>
In this part of algorithm we pass alpha and beta down the tree.
<br>
α: MAX’s best option on path to root
<br>
β: MIN’s best option on path to root
<br>
In minimizer node, if we find a solution that is lower from alpha we don't go to it's n
branch and prune it because we know that it's bas way and won't returns a good result for us.
It means that this branch and node is not a way that chosen by minimax algorithm. <br>
__So alpha beta search doesn't choose diffrent moves from minimax algorithm.__


In [3]:
    def maxValue(self, board, alpha, beta, depth):
        if depth == self.depth:
            return self.evaluate(board)
        maximum = float(-inf)
        moves = self.generateMoves(board, self.side)
        if not moves:
            return self.evaluate(board)
        for move in moves:
            next_board = self.nextBoard(board, self.side, move)
            maximum = max(maximum, self.minValue(next_board, alpha, beta, depth+1))
            if maximum > beta:
                return maximum
            alpha = max(alpha, maximum)
        return maximum


    def minValue(self, board, alpha, beta, depth):
        if depth == self.depth:
            return self.evaluate(board)
        minimum = float(inf)
        moves = self.generateMoves(board, self.opponent(self.side))
        if not moves:
            return self.evaluate(board)
        for move in moves:
            next_board = self.nextBoard(board, self.opponent(self.side), move)
            minimum = min(minimum, self.maxValue(next_board, alpha, beta, depth+1))
            if minimum < alpha:
                return minimum
            beta = min(beta, minimum)
        return minimum

### Average Time comparision between alpha-beta VS. minimax search
We can see each move time average for each of this algorithm is below table. <br>
Alpha beta prunning prunes state space and save a lot of time as you see in below table. <br>
By this table and 5 second time limit we can set minimax maximum depth to 3(almost 4) and for alpha beta version set it to 5.

<img src="images/comparision.png">

### evaluate(self, board)
This function evaluates each state utillity(How much is good for win). We use depth-limited search, so our algorithm ends up in a non terminal state. So we can't use main utillity and we shoud use an estimation for it. This estimation has been done by __evaluate__ funciton. <br>
Game is over when one player can't move. So i think to two criterias: <br>
1. How many moves that we can make and how many our opponent can make
    + It's a good measure somehow because how much is lower we have more risk to lose the game.
    + I try diffrence, weighted diffrence and weighted ratio but agent with simple ratio wins more games against this agents. So i choose ratio of this two parameter.
2. How many movable pices i have versus how many movable pieces my opponent have
    + It's a good measure somehow because how much is upper, we have more chance to win the game.
    + I try diffrence, weighted diffrence and weighted ratio but agent with simple ratio wins more games against this agents. So i choose ratio of this two parameter.
I compete this two types of agents against each other and see that in some situations first wins and in other situations second one wins. So i decide to combine this two parameter with some coefficient to get better evaluation for states.
I see this pattern that in board sizes lower than 5, minimax wins my agent. So i decide to make board size a parameter in my evaluation function.

In [5]:
    def evaluate(self, board):
        my_moves = self.generateMoves(board, self.side)
        my_movable_pieces = {(move[0], move[1]) for move in my_moves}
        if not my_moves:
            return -1000 # Loosing point
        opponent_moves = self.generateMoves(board, self.opponent(self.side))
        opp_movable_pieces = {(move[0], move[1]) for move in opponent_moves}
        if not opponent_moves:
            return 1000 # Winning point
        first_eval = len(my_moves) / len(opponent_moves)
        second_eval = len(my_movable_pieces)/len(opp_movable_pieces)
        return first_eval + 2 * second_eval